In [20]:
"""
Created on Mon Nov 27 14:05:26 2019

@author: usingh
This notebook covers few example use cases of pyrpipe
"""

#import pyrpipe class
from pyrpipe import sra

In [21]:
#define environment to work with
testDir="/home/usingh/work/urmi/hoap/test" #all the downloads and results will be under this directory
#Specify hisat index to be used later
hisatInd="/home/usingh/work/urmi/hoap/test/hisatYeast/S288C_reference_genome_R64-2-1_20150113/yeastIndex"
#small list of yeast RNA-Seq runs
yeastList=['SRR1583780','SRR5507495','SRR5507442','SRR5507362','SRR5507343','SRR5507356','SRR5507413']

## Creating SRA objects

We will use the SRA class to create objects which will refer to an "SRR" sample (also knonw as the Run object see: https://www.ncbi.nlm.nih.gov/books/NBK56913/#search.what_do_the_different_sra_accessi). We need to create an SRA object for each run we want to analyze. We will specify a directory (testDir) which will be used to store all the data. By default this directory is the current working directory. To make things consistent, for each run
all data will be downloaded under the sub-directory &lt;SRRaccession> (this is enforced behaviour).

In [22]:
#let's consider the run SRR1583780
myRun='SRR1583780'
'''
create SRA object. Note that the test directory parameter specifies where to put all the data. 
By default the directory is the current working directory. To make things consistent, for each run
all data will be downloaded under the sub-directory <SrrAccession>.
'''
sraOb=sra.SRA(myRun,testDir)
#get the accession
sraOb.getSrrAccession()
#check the location
print("Location is: "+sraOb.location)
#check if sra file exists
sraOb.sraFileExistsLocally()


Creating SRA: SRR1583780
Location is: /home/usingh/work/urmi/hoap/test/SRR1583780


False

## Downloading SRA file
In the above chunk we have created an SRA object. ```sraFileExistsLocally()``` checks whether the .sra files exist locally (&lt;SRRaccession>.sra). To download .sra files using ```prefetch``` we can use ```downloadSRAFile()```

In [23]:
sraOb.downloadSRAFile()

if sraOb.sraFileExistsLocally():
    print("sra file downloaded to"+sraOb.localSRAFilePath)
else:
    raise Exception("Error Downloading")

Executing:prefetch -O /home/usingh/work/urmi/hoap/test/SRR1583780 SRR1583780


2019-11-27T20:38:17 prefetch.2.9.3: 1) 'SRR1583780' is found locally

Downloaded file: /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780.sra 215.3 KB 
sra file downloaded to/home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780.sra


## Converting sra to fastq
Once .sra file is downloaded we can use ```runFasterQDump()``` to conver .sra to .fastq files. The fastq files will be created in the same directory (```sraOb.location```)


In [24]:
'''
run fasterq-dump.
deleteSRA arguments tells to remove the .sra file after getting the .fastq files
using **kwags we can pass additional arguments to our program. These are separated by key-value pairs just like
regular linux argument. The flags (options without arguments) should be passed as having empty value i.e. "" 
(e.g. -f below)
'''
sraOb.runFasterQDump(deleteSRA=True,**{"-f":"","-t":testDir})

Executing:fasterq-dump -f -t /home/usingh/work/urmi/hoap/test -O /home/usingh/work/urmi/hoap/test/SRR1583780 -o SRR1583780.fastq /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780.sra
spots read      : 4,881

reads read      : 9,762

reads written   : 9,762

Deleting file: /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780.sra rm /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780.sra


True

## Quality control and adapter filtering
To remove adapters and filter reads from fastq files we can use ```performQC()``` function. The ```performQC()``` function takes an object of type ```RNASeqQC``` which are basically programs to perform quality filtering on RNA-Seq. So far ```trim_galore``` and ```BBMap``` are implemented in pyrpipe. To run ```performQC()```, we have to first create an ```RNASeqQC``` object.


In [28]:
'''
First we need to import qc which contains all RNASeqQC objects.
create TrimGalore object to use trim_galore. Note TrimGalore inherits RNASeqQC class.
Note the **kwargs passed to TrimGalore
'''
from pyrpipe import qc
trimGaloreObj=qc.Trimgalore(**{"-j":"8","--length":"1"})
#now perform qc
sraOb.performQC(trimGaloreObj,deleteRawFastq=False)

Checking trim_galore...
Found trim_galore
Checking cutadapt...
Found cutadapt
['-j', '8', '--length', '1']
Performing QC using trim_galore
Running trim_galore paired
Executing: trim_galore -o /home/usingh/work/urmi/hoap/test/SRR1583780 -j 8 --length 1 --paired /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_1.fastq /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_2.fastq
Executing:mv /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_1_val_1.fq /home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_1_trimGalore.fastq
(True, '/home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_1_trimGalore.fastq', '/home/usingh/work/urmi/hoap/test/SRR1583780/SRR1583780_2_trimGalore.fastq')


True